## current

In [1]:
# ============================================================
#  Step 0/8 : Colab Environment Setup
# ============================================================
# ⚠️  This block is intended for Google Colab development only.
#     When running as CLI (grader.py), this section will be skipped.
print("🔹 Step 0/8: Installing minimal dependencies and setting up environment...")

# ------------------------------------------------------------
# 0.1 Mount Google Drive
# ------------------------------------------------------------
from google.colab import drive

try:
    drive.mount("/content/drive")
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    print(f"⚠️ Drive mount skipped or failed: {e}")

# ------------------------------------------------------------
# 0.2 Dependency check & install
# ------------------------------------------------------------
print("📦 Checking required libraries...")
!pip install -q nbformat==5.10.4 pandas==2.2.2

# ------------------------------------------------------------
# 0.3 Suppress warnings (Deprecation, Future)
# ------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("🔇 Deprecation and Future warnings are suppressed.")

# ------------------------------------------------------------
# 0.4 Set working directory
# ------------------------------------------------------------
import os, sys
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"

try:
    os.chdir(PKG_PARENT)
    sys.path.append(PKG_PARENT)
    print(f"📁 Working directory set to: {PKG_PARENT}")
except Exception as e:
    print(f"⚠️ Directory change failed: {e}")


🔹 Step 0/8: Installing minimal dependencies and setting up environment...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.
📦 Checking required libraries...
🔇 Deprecation and Future warnings are suppressed.
📁 Working directory set to: /content/drive/MyDrive/Colab Notebooks/autograder/src


In [2]:
# ============================================================
#  Step 1/8 : Importing libraries & initializing
# ============================================================
print("🔹 Step 1/8: Importing libraries & initializing...")

# ------------------------------------------------------------
# 1.1. Standard & common libraries
# ------------------------------------------------------------
import os
import re
import unicodedata
from pathlib import Path
from difflib import SequenceMatcher
import datetime as dt

import nbformat
import pandas as pd


# ------------------------------------------------------------
# 1.2. AutoGrader internal modules
# ------------------------------------------------------------
from autograder.policy import (
    BASE_SCORE,
    PENALTY_REQUIRED_MISS,
    PENALTY_REQUIRED_MISMATCH,
    PENALTY_OPTIONAL_MISS,
    outputs_equal,
    decide_status_and_match,
)

from autograder.io_utils import (
    KST,
    now_kst,
    extract_id_and_name,
    mtime_kst,
    load_config,
)

from autograder.nb_utils import (
    LABEL_PATTERN,
    _normalize_code,
    _extract_label,
    _label_map,
    _nb_fingerprint,
    _cell_output_text,
    _indexes_with_labels,
    _sim,
    _label_key_robust,
)

from autograder.report import (
    build_stats_block,
    build_excluded_summary_line,
    render_run_summary,
    build_run_log_lines,
)

from autograder.label_tagging import (
    template_label_tagging,
    classify_labels,
)

from autograder.similarity import compute_similarity_pairs


# ------------------------------------------------------------
# 1.3. Load session config & define key paths
# ------------------------------------------------------------
# Select session (차시) & mode
session = load_config("autograder/configs/sessions.toml", 9, "DEV")
# session = load_config("autograder/configs/sessions.toml", 9, "PROD")

# Core paths
TEMPLATE_PATH = Path(session["template_path"])   # 템플릿
ANSWER_PATH   = Path(session["answer_path"])     # 정답
SUBMIT_DIR    = Path(session["submit_dir"])      # 제출파일들
OUT_DIR       = Path(session["out_dir"])         # Autograde 출력 루트

# Tagged template & audit
TAGGED_TEMP_PATH = OUT_DIR / "tagged_template.ipynb"
TAG_AUDIT_PATH   = OUT_DIR / "tag_audit.csv"

# Execution output folder
RUN_TS   = now_kst().strftime("%Y%m%d_%H%M%S")
EXEC_DIR = OUT_DIR / "executed" / RUN_TS

OUT_DIR.mkdir(parents=True, exist_ok=True)
EXEC_DIR.mkdir(parents=True, exist_ok=True)


# ============================================================
#  Step 2/8 : 템플릿 레이블 태깅
# ============================================================
print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")

if not Path(TAGGED_TEMP_PATH).exists():
    template_label_tagging(TEMPLATE_PATH, ANSWER_PATH, TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
    print("✅ Tagged template created!")
else:
    print("ℹ️ Tagged template already exists.")

# ============================================================
#  Step 3/8 : 태깅된 템플릿으로부터 레이블 정보 획득
# ============================================================
print("\n🔹 Step 3/8: Reading tagged template and answer (label-based)...")

req_labels, opt_labels, req_idx, opt_idx, template_fp = classify_labels(TAGGED_TEMP_PATH)

# ============================================================
#  Step 4/8 : 옵션/임계값 및 이전 결과 로드
# ============================================================
print("\n🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")

TEMPLATE_SIM_THRESHOLD = 0.98
PAIR_SIM_THRESHOLD = 0.99
ENABLE_SIMILARITY_CHECK = False   # False로 바꾸면 Step 6 스킵


prev_ids = set()
prev_files = sorted(
    Path(OUT_DIR).glob("summary_static_with_name_*.csv"),
    key=lambda p: p.stat().st_mtime
)

if prev_files:
    try:
        prev_df = pd.read_csv(prev_files[-1], dtype=str)
        if "student_id" in prev_df.columns:
            prev_ids = set(prev_df["student_id"].astype(str))
    except Exception:
        prev_ids = set()

# ============================================================
#  Step 5/8 : 제출물 처리 (채점)
# ============================================================
print("\n🔹 Step 5/8: Collecting and grading submissions...")

paths = sorted(Path(SUBMIT_DIR).rglob("*.ipynb"))
rows, today_rows_tmp = [], []
fps, sid2file, sid2name, sid2path = {}, {}, {}, {}

EXCLUDED_REQ_ALL, EXCLUDED_OPT_ALL = set(), set()

def _label_key(x):
    try:
        return [int(t) for t in x.split(".")]
    except Exception:
        return [999999]

def _exec_and_out_expected(stu_map, ans_map, lab):
    """
    expected_output: 정답 셀 출력 유무(True/False).
    - expected_output=True  → 학생 출력(stu_out)이 있어야 '실행' 인정
    - expected_output=False → 채점 제외
    반환: (executed: bool, stu_out_text: str, expected_output: bool)
    """
    sinfo = stu_map.get(lab)
    anscell = ans_map.get(lab, {}).get("cell")
    ans_out = _cell_output_text(anscell) if anscell else ""
    expected_output = bool(ans_out)

    if not sinfo:
        return False, "", expected_output

    scell = sinfo["cell"]
    stu_out = _cell_output_text(scell)
    executed = bool(stu_out) if expected_output else (
        bool(stu_out) or (scell.get("execution_count") not in (None, 0))
    )
    return executed, stu_out, expected_output


# 템플릿 및 정답 로드
tagged_nb = nbformat.read(TAGGED_TEMP_PATH, as_version=4)
ans = nbformat.read(ANSWER_PATH, as_version=4)

# 템플릿/정답/태깅본 제외
skip_names = {TEMPLATE_PATH.name, ANSWER_PATH.name, TAGGED_TEMP_PATH.name}

# ===== 실제 채점 루프 =====
for p in paths:
    if p.name in skip_names:
        continue

    sid, name = extract_id_and_name(p)
    sid2name[sid], sid2path[sid] = name, p

    try:
        nb = nbformat.read(p, as_version=4)
    except Exception as e:
        rows.append([sid, name, p.name, 0.0, "ERROR", "ipynb 파싱 실패", "ERROR", f"노트북 파싱 실패: {e}"])
        if mtime_kst(p).date() == now_kst().date():
            today_rows_tmp.append(rows[-1])
        continue

    # 템플릿 유사도 0점 처리
    fp = _nb_fingerprint(nb)
    fps[sid], sid2file[sid] = fp, p.name
    if _sim(fp, template_fp) >= TEMPLATE_SIM_THRESHOLD:
        rows.append([sid, name, p.name, 0.0, "ZERO", "템플릿과 거의 동일(원본/무변경)", "ZERO",
                     "템플릿과 거의 동일하여 0점 처리되었습니다."])
        if mtime_kst(p).date() == now_kst().date():
            today_rows_tmp.append(rows[-1])
        continue

    # 라벨 맵
    stu_lmap, ans_lmap = _label_map(nb), _label_map(ans)

    # 필수/옵션 라벨 채점
    req_missing, req_mismatch, opt_missing = [], [], []
    excluded_req, excluded_opt = [], []

    for lab in sorted(req_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_req.append(f"#{lab}")
            continue
        anscell = ans_lmap.get(lab, {}).get("cell")
        ans_out = _cell_output_text(anscell) if anscell else ""
        if not executed:
            req_missing.append(f"#{lab}")
            continue
        if not outputs_equal(stu_out, ans_out):
            req_mismatch.append(f"#{lab}")

    for lab in sorted(opt_labels, key=_label_key):
        executed, stu_out, expected_output = _exec_and_out_expected(stu_lmap, ans_lmap, lab)
        if not expected_output:
            excluded_opt.append(f"#{lab}")
            continue
        if not executed:
            opt_missing.append(f"#{lab}")

    # 감점 계산
    score = BASE_SCORE \
        - PENALTY_REQUIRED_MISS * len(req_missing) \
        - PENALTY_REQUIRED_MISMATCH * len(req_mismatch) \
        - PENALTY_OPTIONAL_MISS * len(opt_missing)

    # 피드백 구성
    feedback_lines = []
    if req_missing:
        feedback_lines.append(f"[필수 미실행 {len(req_missing)}개] 셀: {', '.join(req_missing)}")
    if req_mismatch:
        feedback_lines.append(f"[필수 출력 불일치 {len(req_mismatch)}개] 셀: {', '.join(req_mismatch)}")
    if opt_missing:
        feedback_lines.append(f"[연습 미실행 {len(opt_missing)}개] 셀: {', '.join(opt_missing)}")

    status, output_match = decide_status_and_match(req_missing, req_mismatch)
    reason = "" if status == "OK" else "필수 미실행/불일치 존재"

    score_line = (
        f"(채점) base={BASE_SCORE}"
        f"{' − ' + str(PENALTY_REQUIRED_MISS)    + '×' + str(len(req_missing))  + '(필수 미실행)' if req_missing else ''}"
        f"{' − ' + str(PENALTY_REQUIRED_MISMATCH)+ '×' + str(len(req_mismatch)) + '(필수 불일치)' if req_mismatch else ''}"
        f"{' − ' + str(PENALTY_OPTIONAL_MISS)    + '×' + str(len(opt_missing))  + '(연습 미실행)' if opt_missing else ''}"
        f" = {score}"
    )

    feedback = "\n".join(feedback_lines) if feedback_lines else "정상 제출로 판단되었습니다. 수고했습니다!"
    feedback = f"{feedback}\n{score_line}".strip()

    rows.append([sid, name, p.name, score, status, reason, output_match, feedback])

    if mtime_kst(p).date() == now_kst().date():
        today_rows_tmp.append([sid, name, p.name, score, status, reason, output_match, feedback])

    EXCLUDED_REQ_ALL.update(excluded_req)
    EXCLUDED_OPT_ALL.update(excluded_opt)


# ============================================================
#  Step 6/8 : 유사도 비교
# ============================================================
if ENABLE_SIMILARITY_CHECK:
    print("\n🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
    pairs, df_sim = compute_similarity_pairs(
        fps=fps,
        sid2file=sid2file,
        sid2path=sid2path,
        sid2name=sid2name,
        sim_func=_sim,
        threshold=PAIR_SIM_THRESHOLD
    )

    # 저장
    if not df_sim.empty:
        df_sim.to_csv(EXEC_DIR / f"similar_pairs_{RUN_TS}.csv", index=False, encoding="utf-8-sig")
        df_sim.to_csv(Path(OUT_DIR) / "similar_pairs_latest.csv", index=False, encoding="utf-8-sig")
else:
    print("\n⏩ Step 6/8: Similarity check skipped (ENABLE_SIMILARITY_CHECK=False).")
    pairs, df_sim = [], pd.DataFrame()



# ============================================================
#  Step 7/8 : 출력물 저장
# ============================================================
print("\n🔹 Step 7/8: Saving outputs...")

summary_df = pd.DataFrame(
    rows,
    columns=["student_id", "student_name", "file", "score", "status", "reasons", "output_match", "feedback"]
)

# 파일명 구성
summary_filename_ts = f"summary_static_with_name_{RUN_TS}.csv"
similar_filename_ts = f"similar_pairs_{RUN_TS}.csv"
newtoday_filename_ts = f"new_today_{RUN_TS}.csv"

summary_latest = Path(OUT_DIR) / "summary_static_with_name_latest.csv"
similar_latest = Path(OUT_DIR) / "similar_pairs_latest.csv"
newtoday_latest = Path(OUT_DIR) / "new_today_latest.csv"

# CSV 저장
summary_path_ts = EXEC_DIR / summary_filename_ts
summary_df.to_csv(summary_path_ts, index=False, encoding="utf-8-sig")
summary_df.to_csv(summary_latest, index=False, encoding="utf-8-sig")

if pairs:
    df_sim = pd.DataFrame(
        pairs,
        columns=[
            "student_a", "name_a", "file_a", "mtime_a_kst", "size_a_bytes",
            "student_b", "name_b", "file_b", "mtime_b_kst", "size_b_bytes",
            "similarity"
        ]
    )
    df_sim.to_csv(EXEC_DIR / similar_filename_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(similar_latest, index=False, encoding="utf-8-sig")

df_today = pd.DataFrame(today_rows_tmp, columns=summary_df.columns)
df_today.to_csv(EXEC_DIR / newtoday_filename_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(newtoday_latest, index=False, encoding="utf-8-sig")

# ============================================================
#  Step 8/8 : 요약 및 실행 로그
# ============================================================
print("\n🔹 Step 8/8: Logging summary...")

prev_ids = prev_ids or set()
new_ids = set(summary_df["student_id"].astype(str)) - prev_ids if prev_ids else set(summary_df["student_id"].astype(str))

STATS_BLOCK = build_stats_block(summary_df)

# CONFIG / 로그 저장

CONFIG = {
    # ── Run / Time Info ─────────────────────────────────────────────
    "TODAY_DATE": now_kst().date(),
    "KST_NOW": now_kst().strftime("%Y-%m-%d %H:%M:%S KST"),
    "TIMEZONE": "KST (UTC+9)",
    "RUN_TS": RUN_TS,

    # ── Directories (as str) ────────────────────────────────────────
    "OUT_DIR": str(OUT_DIR),
    "EXEC_DIR": str(EXEC_DIR),
    "SUBMIT_DIR": str(SUBMIT_DIR),

    # ── File basenames (name only) ──────────────────────────────────
    "TEMPLATE_FILE": Path(TEMPLATE_PATH).name,
    "ANSWER_FILE": Path(ANSWER_PATH).name,
    "TAGGED_TEMP_FILE": Path(TAGGED_TEMP_PATH).name,
    "TAG_AUDIT_FILE": Path(TAG_AUDIT_PATH).name,
    "RUN_LOG_FILE": "autograde_run.log",
    "SUMMARY_FILE_LATEST": Path(summary_latest).name,
    "SIMILAR_FILE_LATEST": (Path(similar_latest).name if pairs else "N/A"),
    "NEWTODAY_FILE_LATEST": Path(newtoday_latest).name,


    # ── Full paths (as str) ─────────────────────────────────────────
    "TEMPLATE_PATH": str(TEMPLATE_PATH),
    "ANSWER_PATH": str(ANSWER_PATH),
    "TAGGED_TEMP_PATH": str(TAGGED_TEMP_PATH),

    # ── Scoring / Thresholds ────────────────────────────────────────
    "SIM_THRESHOLD_TEMPLATE": TEMPLATE_SIM_THRESHOLD,
    "SIM_THRESHOLD_PAIR": PAIR_SIM_THRESHOLD,
    "SCORE_RULE": "base=100; required: miss -1.0, mismatch -0.5; optional: miss -0.2",

    # ── Required/Optional cells (counts / indexes / maps) ───────────
    "REQUIRED_CELL_COUNT": len(req_labels),
    "OPTIONAL_CELL_COUNT": len(opt_labels),
    "REQUIRED_CELL_INDEXES": sorted(list(req_idx)),
    "OPTIONAL_CELL_INDEXES": sorted(list(opt_idx)),
    "REQUIRED_CELL_MAP": _indexes_with_labels(tagged_nb, req_idx),
    "OPTIONAL_CELL_MAP": _indexes_with_labels(tagged_nb, opt_idx),
    "EXCLUDED_REQ_ALL": ", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) \
                          if EXCLUDED_REQ_ALL else "없음",
    "EXCLUDED_OPT_ALL": ", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) \
                          if EXCLUDED_OPT_ALL else "없음",

    # ── Dataset stats ────────────────────────────────────────────────
    "TOTAL_CNT": len(summary_df),
    "NEW_CNT": len(new_ids),
    "TODAY_CNT": len(df_today),
}
log_lines = build_run_log_lines(CONFIG, STATS_BLOCK, new_ids=new_ids)
Path(CONFIG["OUT_DIR"]).mkdir(parents=True, exist_ok=True)
with open(Path(CONFIG["OUT_DIR"]) / "autograde_run.log", "a", encoding="utf-8") as f:
    f.write("\n".join(log_lines) + "\n")

print("🗒️ Log appended to:", CONFIG["RUN_LOG_FILE"])
print(render_run_summary(CONFIG, STATS_BLOCK))


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
ℹ️ Tagged template already exists.

🔹 Step 3/8: Reading tagged template and answer (label-based)...

🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...

🔹 Step 5/8: Collecting and grading submissions...

⏩ Step 6/8: Similarity check skipped (ENABLE_SIMILARITY_CHECK=False).

🔹 Step 7/8: Saving outputs...

🔹 Step 8/8: Logging summary...
🗒️ Log appended to: autograde_run.log
✅ 완료: /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시
🕒 실행시각: 2025-11-04 20:27:58 KST  [KST (UTC+9)]

📦 데이터 요약
  • 전체 채점 학생 수 : 112명
  • 새로 채점한 학생 수 : 0명
  • 오늘 들어온 파일(KST 2025-11-04) : 0건

🗂 산출물/경로
  • 실행 산출물 폴더 : executed/20251104_202752/
  • 제출 폴더(SUBMIT_DIR) : /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/9차시
  • OUT_DIR : /content/drive/MyDrive/Colab Notebooks/2024 마케팅조사론/Autograde/9차시

🧾 최신 결과 파일
  • 요약(SUMMARY)   : summary_static_with_name_latest.csv
  •